# Báo cáo Đồ án Cuối kì - Lập trình Song song
##### Nhóm 9:
##### Đoàn Thị Minh Anh - 22120213
##### Trần Hoàng Kim Ngân - 22120224

## 1. Mô tả bài toán:
### 1.1. Phát biểu bài toán:
- Xác định bài toán phân loại ảnh:  
  - Bài toán: Học đặc trưng không giám sát từ bộ dữ liệu ảnh CIFAR-10 bằng Autoencoder rồi dùng ác đặc trưng đã học để huấn luyện một bộ phân loại SVM cho nhiệm vụ phân loại 10 lớp.
  - Quy trình:
    - Huấn luyện Autoencoder trên toàn bộ ảnh huấn luyện không dùng nhãn để tối thiểu hóa loss MSE. Encoder xuất ra vector đặc trưng có kích thước 8192 cho mỗi ảnh.
    - Dùng encoder đã huấn luyện để trích xuất đặc trưng cho tập huấn luyện và tập kiểm tra, rồi huấn luyện SVM sử dụng các nhãn lớp.
    - Đánh giá hiệu năng SVM.  
  - Mục tiêu: Chứng minh rằng đặc trưng học được từ Autoencoder giúp phân loại hiệu quả và triển khai CUDA để chạy autoencoder trên GPU hỗ trợ tăng tốc đáng kể so với chạy bằng CPU.
- Mô tả động lực tăng tốc GPU:
  - Bài toán đòi hỏi xử lý một lượng lớn phép tính số học, đặc biệt trong các tầng tích chập (Convolution), phép nhân ma trận và lan truyền ngược. Việc triển khai các phép tính này trên CPU theo tuần tự thường rất mất thời gian và kém hiệu quả. Trong khi đó, các phép tính này đều có khả năng song song hóa cao, hoàn toàn có thể triển khai một cách hợp lý để ứng dụng GPU.
  - Bộ dữ liệu CIFAR-10 với 50,000 ảnh train và 10,000 ảnh test đòi hỏi khả năng truy xuất bộ nhớ mạnh mẽ. Tận dụng GPU tốt sẽ giúp hệ thống xử lý dữ liệu hiệu quả hơn, đạt tốc độ huấn luyện nhanh hơn và cải thiện chất lượng đặc trưng thu được cho nhiệm vụ phân loại hình ảnh.
### 1.2. Tổng quan về bộ dữ liệu CIFAR-10:
- Đặc tả tập dữ liệu (kích thước, lớp, phân chia):
  - Kích thước ảnh: 32x32 pixels (RGB).
  - 10 lớp gồm: airplane,automobile, bird, cat, deer, dog, frog, horse, ship, truck.
  - Training set: 50,000 ảnh (5,000 ảnh/lớp).
  - Test set: 10,000 ảnh (1,000 ảnh/lớp).
  - Tổng số ảnh: 60,000 ảnh
  - Format: File nhị phân với giá trị pixel uint8.
- Hiển thị hình ảnh mẫu từ mỗi lớp (visualization):
![Hình minh họa CIFAR-10](https://drive.google.com/uc?export=view&id=1Gy6BtsEWLgvlRHhyh5M7H5PCLy76H9Ss)

- Giải thích các bước tiền xử lý dữ liệu (normalization, format):
  - Chuyển đổi kiểu dữ liệu: Dữ liệu pixel gốc trong file ảnh là uint8, là số nguyên 8-bit không dấu nằm trong khoảng [0,255]. Để tiện cho việc chuẩn hoá các giá trị về khoảng [0,1], ở đây chúng em sẽ chuyển chúng về kiểu float.
  - Chuyển về [0,1]: Sau khi chuyển sang kiểu số thực, các giá trị pixel sẽ được chuẩn hoá về khoảng [0,1]. Việc này giúp mạng nơ-ron hội tụ nhanh hơn và hoạt động ổn định hơn.
### 1.3. Kiến trúc Autoencoder:
- Mô tả kiến ​​trúc mạng bằng sơ đồ

![Hình sơ đồ kiến trúc mạng](https://drive.google.com/uc?export=view&id=1TZZe0q-VkPQI0rTmbzO1prhDyyeocfeE)
- Xác định kích thước và phép biến đổi lớp
| Khối | Lớp | Tham số chính | Input shape | Output shape |
|---|---|---|---|---|
| Input | - | - | (32, 32, 3) | (32, 32, 3) |
| Encoder | Conv2D + ReLU | 256 filters, K=3, pad=1, stride=1 | (32, 32, 3) | (32, 32, 256) |
| Encoder | MaxPool2D | 2×2, stride=2 | (32, 32, 256) | (16, 16, 256) |
| Encoder | Conv2D + ReLU | 128 filters, K=3, pad=1, stride=1 | (16, 16, 256) | (16, 16, 128) |
| Encoder | MaxPool2D | 2×2, stride=2 | (16, 16, 128) | **(8, 8, 128)** |
| Latent | Flatten (khi trích đặc trưng) | 8×8×128 = 8192 | (8, 8, 128) | **(8192, )** |
| Decoder | Conv2D + ReLU | 128 filters, K=3, pad=1, stride=1 | (8, 8, 128) | (8, 8, 128) |
| Decoder | UpSample2D | scale=2 | (8, 8, 128) | (16, 16, 128) |
| Decoder | Conv2D + ReLU | 256 filters, K=3, pad=1, stride=1 | (16, 16, 128) | (16, 16, 256) |
| Decoder | UpSample2D | scale=2 | (16, 16, 256) | (32, 32, 256) |
| Output | Conv2D | 3 filters, K=3, pad=1, stride=1 | (32, 32, 256) | (32, 32, 3) |

- Giải thích cấu trúc bộ mã hóa-giải mã và biểu diễn latent
  - Encoder (downsampling path): Hai lớp Conv+ReLU giúp học đặc trưng cục bộ (biên, texture) còn MaxPool giúp nèn thông tin và tăng tính bất biến theo dịch chuyển. Kết quả cuối encoder là tensor (8, 8, 128). Khi trích đặc trưng để phân loại, tensor này được flatten thành vector 8192 chiều.
  - Latent space: là nút thắt cổ chai chưa thông tin quan trọng nhất để tái tạo ảnh. Latent càng tốt thì ảnh reconstruct càng gần ảnh gốc.
  - Decoder (upsampling path): đối xứng với encoder, dùng UpSample để đảo quá trình downsample và Conv nhằm tái tạo ảnh về (32, 32, 3). Lớp cuối không dùng activation, thườngđể mô hình tự học giá trị tái tạo.
### 1.4. Mục tiêu đồ án:
- Mục tiêu hiệu suất (thời gian luyện tập, mục tiêu
tăng tốc, độ chính xác): Đồ án hướng đến xây dựng một hệ thống trích chọn đặc trưng và phân loại ảnh hiệu quả trên tập dữ liệu CIFAR-10. Các mục tiêu hiệu suất chính gồm:
  - Giảm thời gian huấn luyện bằng cách tận dụng GPU và tối ưu các bước tính toán quan trọng trong Autoencoder.
  - Đạt được tốc độ tăng tốc (speedup) đáng kể so với phiên bản chạy trên CPU.
  - Đạt độ chính xác phân loại thỏa yêu cầu tối hiểu (khoảng 55-65%).
- Mục tiêu học tập kỹ thuật: Đồ án giúp nhóm phát triển kiến thức, kỹ năng liên quan đến học máy và kỹ năng lập trình song song trên GPU, cụ thể:
  - Hiểu và triển khai được kiến trúc Autoencoder cho bài toán học đặc trưng không giám sát.
  - Nắm vững quy trình huấn luyện mô hình học sâu (forward, backward, tối ưu, đánh giá).
  - Thực hành lập trình song song trên GPU bằng CUDA.
  - Tối ưu hóa các tác vụ tính toán như biến đổi tensor, tích chập, nhân ma trận nhằm khai thác tối đa khả năng song song của GPU.
- Tiêu chí thành công
  - Chức năng đúng: Autoencoder huấn luyện thành công, tái tạo ảnh hợp lý và bộ phân loại hoạt động chính xác.
  - Trích chọn đặc trưng hiệu quả: đặc trưng nén từ Autoencoder giúp cải thiện độ chính xác phân loại so với baseline.
  - Tăng tốc rõ rệt khi chuyển từ CPU sang GPU, với báo cáo định lượng (speedup, thời gian huấn luyện).
  - Kết quả thực nghiệm đầy đủ bao gồm biểu đồ loss, accuracy, tốc độ huấn luyện, hình ảnh tái tạo, confusion matrix.
  - Báo cáo hoàn chỉnh, phân tích hợp lý và giải thích được tác động của GPU trong toàn hệ thống.

## 2. Các giai đoạn triển khai:


### 2.1. CPU cơ bản
#### Mục tiêu:
- Mục tiêu nhóm muốn đạt được trong giai đoạn này:
  - Xây dựng một phiên bản hoàn chỉnh của autoencoder trên CPU để làm baseline tham chiếu cho các phiên bản GPU sau này.
  - Triển khai đầy đủ các layer cần thiết (Convolution, ReLU, MaxPooling, Upsampling, MSE Loss) cùng forward và backward pass trên CPU để đảm bảo mô hình có thể train được và đạt được reconstruction loss hợp lý.
  - Đo lường thời gian huấn luyện và hiệu suất ban đầu trên CPU, từ đó xác định rõ các điểm nghẽn hiệu năng (bottleneck) để làm cơ sở cho việc tối ưu hóa trên GPU ở các phase tiếp theo.
  - Có thể tạo ra một phiên bản đúng và có thể chạy được để dễ dàng so sánh kết quả với các phiên bản GPU khác và làm nền tảng cho việc chuyển đổi sang code CUDA.
- Tại sao giai đoạn này là cần thiết:
  - Kiểm chứng độ chính xác: Lập trình song song trên GPU vốn phức tạp với các vấn đề về đồng bộ luồng và quản lý bộ nhớ. Khi kết quả đầu ra bị sai lệch, rất khó để phân định nguyên nhân đến từ lỗi logic thuật toán hay lỗi kỹ thuật song song. Phiên bản CPU hoạt động như một chân lý (Ground Truth) tin cậy. Trong các giai đoạn sau, kết quả từ GPU sẽ được so sánh trực tiếp với CPU để đảm bảo tính đúng đắn trước khi tính đến hiệu năng.
  - Nhận diện điểm nghẽn: Việc chạy trên CPU giúp xác định lớp nào (Convolution, Pooling hay Upsampling) chiếm nhiều thời gian xử lý nhất. Đây là chỉ dẫn quan trọng để biết nên dồn lực tối ưu vào lớp nào khi chuyển sang GPU.
  - Thấu hiểu sâu sắc kiến trúc giải thuật: Việc chuyển đổi trực tiếp một công thức toán học sang CUDA kernel là rất rủi ro nếu chưa hiểu rõ luồng dữ liệu. Việc tự tay cài đặt các lớp phức tạp như Convolution hay MaxPooling bằng các vòng lặp tường minh giúp lập trình viên nắm bắt tường tận cơ chế truy cập bộ nhớ, cách xử lý biên và sải bước (stride).
#### Chi tiết triển khai:
- Cách tải và tiền xử lý dữ liệu CIFAR-10:

  Chúng em tải bộ dữ liệu CIFAR-10 dạng binary nguyên bản từ trang web chính thức của tác giả.

  Bộ dữ liệu gồm 5 file training batch (`data_batch_1.bin` đến `data_batch_5.bin`) và 1 file test batch (`test_batch.bin`).

  Mỗi bản ghi trong các file có đúng *3073* bytes, được sắp xếp như sau:  
  - Byte 0: nhãn lớp (1 byte, giá trị 0-9).
  - Byte 1-1024: toàn bộ kênh màu đỏ (1024 pixel).
  - Byte 1025-2048: toàn bộ kênh màu xanh lá (1024 pixel).
  - Byte 2049-3072: toàn bộ kênh màu xanh lam (1024 pixel).

  Quy trình đọc và tiền xử lý dữ liệu được thực hiện như sau:

  - Mở và đọc lần lượt từng file binary theo đúng thứ tự.
  - Với mỗi bản ghi 3073 bytes, tách riêng 1 byte nhãn và 3072 byte pixel.
  - Chuyển ngay 3072 byte pixel từ kiểu `uint8` sang `float` trong quá trình đọc, đồng thời giữ nguyên thứ tự kênh R, G và B như file gốc.
  - Lưu toàn bộ dữ liệu ảnh train (50.000 ảnh) và test (10.000 ảnh) vào các mảng float liên tục trong bộ nhớ host với kích thước là *N * 3072*.
  - Sau khi tải xong, thực hiện chuẩn hóa một lần duy nhất bằng cách chia toàn bộ giá trị pixel cho 255.0 để đưa về khoảng [0.0, 1.0].
  - Để hỗ trợ mini-batch training, chúng em duy trì một mảng chỉ số `train_indices[50000]`. Ở đầu mỗi epoch, hàm `shuffle_cifar10()` sử dụng thuật toán Fisher-Yates shuffle để xáo trộn ngẫu nhiên thứ tự ảnh.
  - Hàm `get_next_batch()` sao chép nhanh dữ liệu batch từ `train_images` vào bộ nhớ đích bằng `memcpy`, đảm bảo hiệu suất cao và dễ dàng chuyển đổi sang GPU ở các phase sau.
- Mô tả cách triển khai các lớp (Convolution, ReLU, MaxPool, UpSample):
  1) Convolution:
  - Forward: Hàm `Conv2D_Forward` thực hiện phép tích chập chuẩn với padding = 1, stride = 1, kernel 3x3. Duyệt theo thứ tự NCHW, hỗ trợ nhiều input và output channels. Bias được cộng trực tiếp sau tích chập.
  - Backward: Tương tự với phiên bản forward, backward cũng thực hiện duyệt mảng theo thứ tự NCHW.
    - Input: Hàm `Conv2D_Backward_Input` thực hiện full convolution giữa gradient từ lớp sau (d_output) với kernel đã được xoay 180°. Kết quả được cộng dồn vào đúng vị trí tương ứng trên input gradient.
    - Weight: Hàm `Conv2D_Backward_Kernel` thực hiện tích chập giữa input của lớp hiện tại và gradient từ lớp sau (d_output), sau đó cộng dồn vào ma trận gradient trọng số.
    - Bias: Hàm `Conv2D_Backward_Biases` thực hiện với mỗi filter, gradient của bias bằng tổng tất cả các giá trị gradient tương ứng trên toàn bộ feature map đầu ra (d_output).
  2) ReLU:
  - Forward: Với từng phần tử, thực hiện so sánh giá trị input[i], với i là index hiện tại trong mảng đầu vào, với 0.0f nhằm đảm bảo đầu ra luôn dương.
  - Backward: Hàm `Relu_Backward` nhân gradient từ lớp sau với 1 nếu input > 0, ngược lại bằng 0, đảm bảo gradient chỉ lan truyền qua các neuron dương.
  3) MaxPool:
  - Forward: Hàm `MaxPool2D_Forward` thực hiện pooling 2x2, stride = 2, giảm kích thước không gian xuống một nửa. Với mỗi vị trí output, tìm giá trị lớn nhất trong vùng 2x2 của từng kênh.
  - Backward: Hàm `MaxPool2D_Backward` chỉ truyền gradient về đúng vị trí có giá trị max trong forward pass và các vị trí còn lại nhận giá trị 0.
  4) UpSample:
  - Forward: Hàm `UpSample2D_Forward` thực hiện nearest-neighbor upsampling với scale_factor = 2. Mỗi pixel đầu vào được sao chép vào khối 2x2 tương ứng trên output.
  - Backward: Hàm `UpSample2D_Backward` cộng dồn gradient từ toàn bộ khối 2x2 của output về đúng pixel gốc trên input.
- Vòng lặp huấn luyện được cấu trúc như sau:
  1) Lặp qua các epoch: Chúng em huấn luyện trong 20 epoch.
  2) Xáo trộn dữ liệu: Ở đầu mỗi epoch, hàm `shuffle_cifar10(&data)` được gọi để áp dụng thuật toán Fisher-Yates shuffle trên mảng `train_indices`, đảm bảo thứ tự các ảnh được thay đổi ngẫu nhiên, giúp mô hình học tốt hơn và tránh bias theo thứ tự file.
  3) Xử lý từng mini-batch:
  - Số batch mỗi epoch được tính tự động bằng công thức: `num_batches = train_subset_size / batch_size`, ở đây do có những giới hạn về bộ nhớ, nên chúng em chỉ gán `train_subset_size = 1000` (chỉ train với 1000 ảnh). Ngoài ra, nhóm áp dụng `batch_size = 32`.
  - Với mỗi batch_id, hàm `get_next_batch()` sao chép nhanh dữ liệu từ `train_images` (theo thứ tự đã shuffle) vào bộ nhớ tạm `batch_images` bằng `memcpy`.
  - Dữ liệu batch được copy vào batch_input của struct autoencoder (buffer đã được cấp sẵn trong struct CPUAutoEncoder) để các hàm forward/backward có thể truy cập trực tiếp.
  4) Quy trình huấn luyện một batch:
  - Forward pass: Gọi hàm dành cho forward để tính lần lượt output của tất cả các layer từ input đến reconstructed output.
  - Tính loss: Sử dụng hàm để tính Mean Squared Error giữa ảnh gốc (batch_input) và ảnh tái tạo (final_output). Loss của batch hiện tại được hiển thị trên terminal.
  - Backward pass: Gọi hàm dành cho backward để tính gradient cho tất cả các layer theo thứ tự ngược lại.
  - Cập nhật tham số: Gọi hàm cập nhật để áp dụng Gradient Descent với `learning rate = 0.001`, cập nhật trọng số và bias của tất cả các lớp Convolution.
- Một vài đoạn mã chính:
  1) Cấu trúc hàm Conv2D():
  ```c
  void Conv2D_Forward(float* input, int input_width, int input_height, int input_channels,
    float* kernel, int kernel_width, int kernel_height,
    float* biases, int padding, int stride, int filter_count,
    float* output, int output_height, int output_width) 
  {
      // Lặp qua kênh đầu ra (filter)
      for (int c_out = 0; c_out < filter_count; c_out++) {  
          // Lặp qua chiều cao output
          for (int h_out = 0; h_out < output_height; h_out++) {
              // Lặp qua chiều rộng output
              for (int w_out = 0; w_out < output_width; w_out++) {
                  // Lặp qua kênh đầu vào (c_in)
                  for (int c_in = 0; c_in < input_channels; c_in++) {
                      // Lặp qua kernel height
                      for (int k_h = 0; k_h < kernel_height; k_h++) {
                          // Lặp qua kernel width
                          for (int k_w = 0; k_w < kernel_width; k_w++) {
                              // Tính toán vị trí input tương ứng. 
                              // Kiểm tra zero padding.
                              // Tính toán vị trí trọng số.
                              // Thực hiện phép tích chập.
                              // Với val là giá trị trong mảng input (có thể là 0.0f do thuộc phần padding), kernel: mảng trọng số, 
                              weight_idx: vị trí giá trị trọng số trong mảng kernel.
                              sum += val * kernel[weight_idx];
                          }
                      }
                  } 
                  sum += biases[c_out];  // Thêm bias
                  // Gán sum cho giá trị pixel đầu ra.
                  output[output_idx] = sum;
              }
          }
      }
  }
  ```
  2) Cấu trúc vòng lặp chính:
  ```c
  // Cấu trúc vòng lặp huấn luyện chính
  for (int epoch = 0; epoch < num_epochs; epoch++) {
      float total_loss = 0.0f;
      clock_t start_time = clock(); // Bắt đầu đo giờ

      // 1. Xáo trộn dữ liệu (Data shuffling)
      shuffle_indices(train_indices, num_train_images);

      for (int batch_id = 0; batch_id < num_batches; batch_id++) {
          // 2. Chuẩn bị dữ liệu batch
          load_batch(autoencoder.input, train_images, train_indices, batch_id, batch_size);

          // 3. Lan truyền xuôi (Forward pass)
          // Dữ liệu đi qua: Conv -> ReLU -> MaxPool -> ... -> Decoder
          forward_autoencoder(&autoencoder);

          // 4. Tính toán Loss (MSE)
          float batch_loss = MSE(autoencoder.input, autoencoder.output, input_size);
          total_loss += batch_loss;

          // 5. Lan truyền ngược (Backward pass)
          // Tính gradient từ output ngược về input
          backward_autoencoder(&autoencoder);

          // 6. Cập nhật tham số (Update weights)
          // W = W - learning_rate * gradient
          update_autoencoder_parameters(&autoencoder, learning_rate);
      }

      // Kết thúc epoch: Tính thời gian thực thi epoch.
      clock_t end_time = clock();
      double epoch_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
  }
  ```
#### Kết quả:
- Thời gian huấn luyện cho mỗi epoch và tổng thời gian huấn luyện:
  - Với mỗi epoch, phiên CPU sẽ thực thi trong khoảng 489 giây.
  - Tổng thời gian thực thi 20 epoch là khoảng 9780 giây (tương đương gần với 2 giờ 43 phút).
- Final reconstruction loss: 0.054788.
- Sample reconstructed images (show original vs reconstructed)
![Ảnh mẫu reconstructed bởi CPU](https://drive.google.com/uc?export=view&id=1HPJctlPGdm7VdnxSwX7QfXSTkSVR500y)
- Memory usage
#### Những điểm chính:
1) Bài học rút ra từ thuật toán:
- Qua quá trình tự cài đặt thủ công các lớp mạng nơ-ron và thuật toán lan truyền ngược (Backpropagation) trên CPU, chúng em đã rút ra những bài học quan trọng về bản chất của thuật toán:
  - Cường độ tính toán cực lớn: Khi trực tiếp cài đặt thuật toán Convolution, chúng em nhận thấy rằng đây không chỉ là phép nhân ma trận đơn thuần mà là một chuỗi các phép tính tích vô hướng (dot product) trượt qua không gian đầu vào. Với mỗi lớp Conv2D, việc áp dụng các bộ lọc (filters) kích thước 3×3 lên toàn bộ chiều sâu của ảnh đầu vào tạo ra một khối lượng tính toán khổng lồ.
  - Cơ chế nén và tái tạo thông tin: Việc cài đặt tuần tự các lớp của Autoencoder giúp nhóm có thể hình dung rõ cách Autoencoder học các đặc trưng.
  - Kiểm soát biên và bộ nhớ: Thao tác trực tiếp với Padding và Stride cho thấy tầm quan trọng của việc tính toán chỉ số (indexing) chính xác. Một sai sót nhỏ tại biên không chỉ làm lệch kích thước Tensor mà còn gây lỗi truy cập bộ nhớ nghiêm trọng.
2) Những hiểu biết sâu sắc đã hướng dẫn việc triển khai GPU:
- Khai thác tính độc lập dữ liệu cho mô hình SIMT: Thông qua việc cài đặt thủ công các lớp Convolution, MaxPool và UpSample, chúng em nhận thấy các phép toán tại mỗi vị trí đầu ra (h,w,c) hoàn toàn độc lập. Giá trị của một pixel trong Feature Map đích không phụ thuộc vào việc tính toán các pixel lân cận. 
  - Định hướng GPU: Đặc tính này hoàn toàn phù hợp với kiến trúc SIMT (Single Instruction, Multiple Threads) của CUDA. Thay vì sử dụng vòng lặp tuần tự, chúng em sẽ thiết kế kernel sao cho mỗi luồng (thread) hoặc một nhóm luồng sẽ chịu trách nhiệm tính toán song song cho một điểm ảnh cụ thể, tận dụng tối đa số lượng nhân CUDA khổng lồ.
- Chiến lược giải quyết nút thắt cổ chai tính toán: Việc quan sát mã nguồn CPU cho thấy lớp Conv2D sở hữu độ phức tạp tính toán lớn nhất với 6 vòng lặp lồng nhau. Đây chính là điểm nghẽn hiệu năng chính, chiếm phần lớn thời gian huấn luyện.
  - Định hướng GPU: Để giải quyết vấn đề này, chiến lược song song hóa cần tập trung vào việc trải phẳng (flatten) các chiều của vòng lặp và ánh xạ chúng vào không gian lưới (Grid/Block) của GPU.
- Tối ưu hóa mô hình truy cập bộ nhớ: Phân tích cơ chế "cửa sổ trượt" (sliding window) của phép tích chập cho thấy một lượng lớn dữ liệu đầu vào bị đọc lặp lại dư thừa. Cụ thể, với kernel 3×3, một pixel đầu vào được truy xuất để tính toán cho 9 pixel đầu ra khác nhau. Trên CPU, việc truy cập bộ nhớ liên tục nhưng tại các địa chỉ rời rạc gây lãng phí băng thông nghiêm trọng.
  - Định hướng GPU: Việc chỉ dựa vào Global Memory trên GPU sẽ tạo ra độ trễ lớn. Nhận định này dẫn dắt nhóm đến việc ứng dụng Shared Memory Tiling. Bằng cách tải các khối dữ liệu (tiles) vào bộ nhớ chia sẻ tốc độ cao, các luồng trong cùng một block có thể tái sử dụng dữ liệu hiệu quả, giảm thiểu tối đa lưu lượng truy cập xuống Global Memory.

### 2.2. GPU cơ bản
#### Mục tiêu:
- Chuyển việc triển khai trên CPU sang trên GPU bằng song song hóa cơ bản cho các phép toán chính là Con2D, MaxPool2D, ReLU và UpSample2D.
- So sánh output GPU với CPU để đảm bảo kết quả đúng so với CPU.
- Thiết lập baseline hiệu suất GPU: đo thời gian thực thi theo kernel và tổng thời gian để làm mốc cho các version tối ưu phía sau.
#### Chi tiết cài đặt:
- Chiến lược song song hóa:
  - Layout dữ liệu: Tensor được lưu dạng 1D liên tục theo NCHW với công thức tỉ số là
  $$idx(n,c,h,w) = ((n \cdot C + c)\cdot H +h)\cdot W +w$$
  - Các kernel được thiết kế theo nguyên tắc:
    - Các phép toán tạo output 4D như Conv/Pool/Upsample: 1 thread xử lý 1 phần tử output.
    - Các phép toán "element-wise" như ReLU: 1 thread xử lý 1 phần tử.
  - Cách ánh xạ thread:
    - `blockIdx.x` và `blockIdx.y` quét theo không gian `w` và `h`.
    - `blockIdx.z` gộp `(N,C)` hoặc `(N, C_out)` bằng chỉ số `nc`.
- Thiết kế kernel:

  **1. Convolution:**
    - Forward: Mỗi thread tính 1 phần tử output tại tọa độ `(n, c_out, h_out, w_out)`. Lưới tính toán được ánh xạ bằng grid 3D với các trục x-y tương ứng với vị trí không gian của output và trục z mã hóa batch và kênh đầu ra. Mỗi thread duyệt toàn bộ kênh vào và kernel `K × K`, tự tính chỉ số input/weight theo định dạng NCHW để thực hiện pháp nhân-cộng. Padding được xử lý bằng kiểm tra biên trực tiếp và kết quả cuối cùng được ghi vào bộ nhớ output theo chỉ số tương ứng (cộng bias nếu có).
    - Backward:
      - Input: Hàm được thiết kế để tính gradient theo input dX, với mỗi thread phụ trách một phần tử `(n, c_in, h, w)` của tensor đầu vào. Thread duyệt toàn bộ `c_out`, kernel `K × K` và kiểm tra điều kiện stride để truy ngược vị trí tương ứng trên dY. Kernel có được xoay 180 độ theo đúng quy tắc convolution gradient. Chỉ số được tính theo định dạng NCHW và kết quả ghi trực tiếp vào dX.
      - Weight: Mỗi thread chịu trách nhiệm tính gradient cho đúng 1 phần tử weight tại vị trí `(c_out, c_in, kh, kw)`. Thread duyệt toàn bộ batch và toàn bộ vị trí `h_out`, `w_out` của output, nhân dY với phần tử input tương ứng rồi cộng dồn vào một biến sum cục bộ.
      - Bias: Hàm được thiết kế theo hướng mỗi thread tính gradient cho một phần tử bias tương ứng c_out. Thread duyệt toàn bộ batch và toàn bộ spatial vị trí của dY, cộng dồn toàn bộ giá trị gradient để tạo ra `dB[c_out]`. 

  **2. MaxPool:**
  - Forward: Kernel `maxpool2x2_forward` thực hiện max pooling 2×2 với `stride = 2`, giảm kích thước từ `H×W` xuống `H/2 × W/2`. Duyệt theo layout NCHW, trong đó mỗi thread phụ trách một phần tử output `(n, c, h_out, w_out)`. Thread ánh xạ về cửa sổ input 2×2 tương ứng `(h_in0=2*h_out, w_in0=2*w_out)`, lấy giá trị lớn nhất trong 4 phần tử và ghi ra output.
  - Backward: Kernel `maxpool2x2_backward` lan truyền gradient qua max pooling bằng cách chỉ chuyển gradient về đúng phần tử đạt max trong cửa sổ 2×2. Với mỗi output `(n, c, h_out, w_out)`, thread đọc 4 giá trị input, xác định `max_idx`, lấy `g = grad_out[...]` rồi gán `grad_in[...] = g` tại vị trí max (các vị trí còn lại giữ 0). Thiết kế giả định `grad_in` đã được zero trước khi chạy.
  
  **3. UpSample:**
  - Forward: Kernel `upsample2x2_forward` thực hiện nearest-neighbor upsampling ×2 theo cả chiều cao và chiều rộng, tăng kích thước từ `H×W` lên `2H×2W`. Duyệt theo layout NCHW, mỗi thread tính một phần tử output `(n, c, h_out, w_out)` và ánh xạ ngược về input bằng phép chia nguyên `h_in = h_out/2, w_in = w_out/2`, sau đó copy giá trị input sang output tương ứng.
  - Backward: Kernel `upsample2x2_backward` lan truyền gradient qua nearest-neighbor upsampling bằng cách cộng dồn gradient từ 4 pixel output (khối 2×2) quay về một pixel input. Mỗi thread phụ trách một phần tử input (n, c, h_in, w_in), lấy tổng grad_out tại các vị trí `(2h_in+dh, 2w_in+dw)` với `dh,dw∈{0,1}`, rồi gán vào `grad_in[idx_in]`.

  **4. ReLU:**
  - Forward: Kernel `relu_forward` thực hiện hàm kích hoạt ReLU theo kiểu element-wise trên một vector/ tensor đã được làm phẳng. Mỗi thread xử lý một phần tử `x[i]`, áp dụng phép cắt ngưỡng `x[i] = max(x[i], 0)` và ghi in-place để giảm bộ nhớ trung gian. Có kiểm tra biên `i < size` để đảm bảo an toàn truy cập.
  - Backward: Kernel `relu_backward` tính gradient của ReLU theo từng phần tử: với mỗi `i`, nếu `x[i] > 0` thì truyền gradient `grad_x[i] = grad_y[i]`, ngược lại gán 0. Thiết kế dùng các con trỏ `__restrict__` để giúp tối ưu truy cập bộ nhớ, và mỗi thread ghi đúng một phần tử `grad_x` nên không có xung đột ghi (race condition).

  **5. MSE loss:**
  - Forward: Kernel `mse_loss_forward` tính tổng sai số bình phương giữa output và target trên toàn bộ tensor. Mỗi thread xử lý một phần tử idx, tính `diff²` và ghi vào shared memory sdata, sau đó thực hiện reduction trong block để cộng dồn. Kết quả mỗi block được cộng vào biến loss toàn cục bằng atomicAdd, nên cần đảm bảo loss đã được khởi tạo về 0 trước khi launch. (cần chia cho size ở ngoài để thu được MSE)
  - Backward: Kernel `mse_loss_backward` tính gradient theo từng phần tử cho MSE: $∂L/∂output[i]=2(output[i]−target[i])/size$. Mỗi thread ghi độc lập `grad_out[i]`, có kiểm tra biên `i < size`, không phát sinh race condition và phù hợp cho lan truyền ngược trong autoencoder.
- Chiến lược quản lý bộ nhớ device: Sử dụng chiến lược cấp phát trước (pre-allocation/static). Toàn bộ bộ đệm trên device cho tham số mô hình (weights/biases), các tensor trung gian (activations) và gradient tương ứng được `cudaMalloc` một lần duy nhất trong hàm `gpu_autoencoder_init()` trước khi bắt đầu huấn luyện. Trong quá trình train, mỗi iteration chỉ cần copy input batch từ host sang device và tái sử dụng các buffer đã cấp phát cho các bước forward/backward, nhờ đó tránh việc cấp phát/giải phóng lặp lại trong vòng lặp huấn luyện.
- Vòng lặp huấn luyện được thiết kế như sau:
  1. Lặp qua các epoch: Mô hình được huấn luyện trong 20 epoch với batch_size là 64 và learning rate là 0.001.
  2. Xáo trộn dữ liệu: Ở đầu mỗi epoch, hàm `shuffle_cifar10(&data)` được gọi để xáo trộn thứ tự ảnh huấn luyện.
  3. Xử lý từng mini-batch:
  - Số batch mỗi epoch được tính theo công thức `num_batches = TRAIN_NUM/batch_size`.
  - Với mỗi `batch_id`, hàm `get_next_batch()` lấy dữ liệu batch hiện tại từ tập train và đưa vào buffer host `h_batch`.
  - Batch trên host sau đó được truyền vào `gpu_autoencoder_forward()`, bên trong sẽ copy H2D vào buffer input đã cấp phát sẵn.
  4. Quy trình huấn luyện một batch:
  - Forward pass: Gọi `gou_autoencoder_forward()` để tính lần lượt output qua toàn bộ các layer và tạo reconstructed output.
  - Loss tái tạo (MSE) được tính giữa ảnh gốc và ảnh tái tạo, giá trị loss của batch được dùng để tích lũy `epoch_loss` và được in log mỗi 100 batch.
  - Backward pass và cập nhật tham số: Gọi `gpu_autoencoder_backward()` để tính gradient theo thứ tự ngược và cập nhật weights/bias bằng Gradient Descent với lr = 0.001. 
#### Results:
- Thời gian huấn luyện mỗi epoch: trung bình khoảng 304.57s (khoảng 5 phút 4 giây)
- Thời gian huấn luyện tổng cộng: 6091.42s (khoảng 101 phút 31 giây)
- Tăng tốc so với CPU baseline: GPU nhanh hơn khoảng 80 lần.
- Một số ảnh gốc và ảnh tái tạo:
![Ảnh mẫu reconstructed bởi GPU naive](https://drive.google.com/uc?export=view&id=1Phzzp3KKKRkMTYVaK5ykaM0lLNFVn5-o)
- Kiểm chứng kết quả của GPU so với CPU: Để kiểm chứng kết quả của các kernel chạy trên GPU, chúng em chạy CPU (baseline) và GPU trên cùng một bộ dữ liệu đầu vào/gradient được khởi tạo trước, sau đó so sánh bằng cách tính loss.
  - Cách làm:
    - Khởi tạo `h_x, h_w, h_b,...` theo công thức cố định để thu được kết quả.
    - Chạy CPU để lấy `cpu_out`.
    - Chạy GPU kernel, copy kết quả về host để lấy `gpu_out`.
    - So sánh `cpu_out` và `gpu_out` bằng hàm `compare_arrays()`.
  - Các thông số tính toán:
    - **max|diff|**: sai khác tuyệt đối lớn nhất.
    - **mean|diff|**: sai khác tuyệt đối trung bình.
    - **RMSE**: căn trung bình bình phương sai khác.
  - Các kernel cần verify: Conv2D bao gồm Forward và Backward (dX, dW, dB), MaxPool2D gồm Forward và Backward và UpSample2D bao gồm Forward, Backward.
  - Kết quả thu được:
  ![Kết quả verify GPU](https://drive.google.com/uc?export=view&id=1XpzRLBKkGTfdEMN3Ehp63HNm6pH0Cu03)
#### Profiling Analysis:
- Basic profiling results:  
![Basic profiling results](https://drive.google.com/uc?export=view&id=1-rGNA7ARLzKWIRgPJbxNEbBjtacYX7LE)
  - `conv2d_backward_input_naive` chiếm 49.49% tổng thời gian GPU, 78,100 lần gọi, 39.882 ms/lần (min 6.93 ms, max 100.45 ms).
  - `conv2d_backward_weight_naive` chiếm 32.11%, 78,100 lần gọi, 25.876 ms/lần (min 3.74 ms, max 54.964 ms).
  - `conv2d_forward_naive` chiếm 11.54% (~726.525 s), 78,100 lần gọi, 9.3025 ms/lần.
  - `conv2d_backward_bias_naive` chiếm 5.76% (~362.393 s), 78,100 lần gọi, 4.6401 ms/lần.
  - Các kernel còn lại rất nhỏ: ReLU/MaxPool/Upsample mỗi cái < 0.3%; `cudaMemcpy HtoD/DtoH` tổng khoảng 0.04%; `sgd_update` và `mse_loss` gần như không đáng kể.
- Nhận diện nút thắt ban đầu: 
  - Toàn bộ nhóm Conv2D (forward + backward) chiếm khoảng 98.9% thời gian GPU. Đây là bottleneck chính của chương trình.
  - Thời gian `cudaMemcpy` rất thấp, chứng tỏ hệ thống không bị nghẽn truyền dữ liệu mà chủ yếu bị giới hạn bởi tính toán và truy cập bộ nhớ global của các kernel conv2d.
  - Hướng tối ưu ưu tiên: thay các kernel conv2d bằng bản tối ưu hơn (sử dụng tiling shared memory,...)
#### Key Takeaways:
- Cái gì nhanh bất ngờ/chậm bất ngờ:
  - Nhanh bất ngờ: Các bước phụ trợ như ReLU, MaxPool, Upsample, MSE loss, SGD update đều chiếm tỷ lệ rất nhỏ, điều này cho thấy chúng đang chạy rất hiệu quả.
  - Chậm bất ngờ: Conv2D backward chiếm áp đảo tổng thời gian GPU, đặc biệt là hàm backward cho input và backward cho weight.
- Cơ hội tối ưu: Ưu tiên tối ưu Conv2D backward: Có thể cân nhắc sử dụng tiling và shared memory, vectorized load/store (float4) nếu dữ liệu phù hợp, unroll kernel 3x3,...


### 2.3. GPU Optimized Implementation - Version 1
#### Optimization Focus:
#### Objectives:
- What specific optimization(s) you targeted
- Expected performance improvement
#### Implementation Details:
- Optimization Techniques Applied
  - Detailed explanation of the optimization (e.g., shared memory tiling)- Why this optimization should help
  - Implementation approach
- Key Code Snippets: Show the optimized kernel or key changes
#### Results:
- Training time comparison with previous version
- Speedup over previous phase (incremental and cumulative)
- Performance metrics (bandwidth utilization, occupancy)
- Sample reconstructed images
![Ảnh mẫu reconstructed bởi GPU version 1](https://drive.google.com/uc?export=view&id=1HxMYgUUDyIy5BUthej5hNJIPSvblqDiS)
- Profiling comparison: before vs after
#### Analysis:
- Why did this optimization work (or not work as expected)?
- What did profiling reveal?
- What's the next bottleneck?
#### Key Takeaways:
- Lessons learned from this optimization
- Applicability to other problems

### 2.4. GPU Optimized - Version 2
#### Tập trung tối ưu: 
Tập trung vào việc tối ưu các hàm Conv2D cả về tính toán thực thi lẫn việc load dữ liệu.
#### Mục tiêu:
- Chiến lược tối ưu hóa nhắm đến:
  - Giảm truy cập vào Global Memory bằng cách cooperative loading (các thread trong block cùng tải tile/halo vào shared memory) và tái sử dụng dữ liệu trong quá trình tính tích chập, hạn chế load lặp lại.
  - Giảm chi phí atomic và tranh chấp (atomic contention) ở bước tính gradient trọng số dW bằng block-level reduction trước khi atomicAdd (chỉ 1 thread/1 kernel element thực hiện atomic thay vì mọi thread).
  - Giảm overhead vòng lặp & tăng ILP bằng #pragma unroll cho kernel 3×3 để compiler tối ưu tốt hơn.
  Tăng hiệu quả tính gradient bias bằng warp-level reduction (`__shfl_down_sync`) thay cho reduction thuần shared memory.
- Kỳ vọng cải thiện hiệu năng:
  - Conv2D forward/backward nhanh hơn nhờ giảm bandwidth pressure lên global memory và tăng reuse từ shared/constant memory.
  - Backward weight cải thiện rõ rệt (thường lớn nhất) do số lần atomicAdd giảm mạnh, từ đó giảm contention và tăng throughput.
  - Backward bias nhanh hơn nhờ warp shuffle giảm sync và giảm truy cập shared.
  - Kỳ vọng tổng thể sẽ giảm đáng kể thời gian mỗi iteration/epoch.
#### Chi tiết cài đặt:
1. Conv2D Forward: Cooperative loading + Unroll
- Kỹ thuật áp dụng:
  - Cooperative loading: cả block cùng tải tile (kèm halo) vào shared memory theo tid (mỗi thread load 1–2 phần tử).
  - Unrolling (K=3): `#pragma unroll` cho vòng lặp kernel 3×3.
- Vì sao giúp nhanh hơn:
  - Giảm số lần đọc global memory (dữ liệu input được tái sử dụng trong shared).
  - Tăng coalescing khi load theo nhóm thread.
  - Unroll giảm overhead loop và tăng ILP.
- Cách triển khai:
  - Tính `row_start/col_start` theo block + padding.
  - Vòng `for (idx = tid; idx < total; idx += num_threads)` để load shared.
  - Compute convolution từ shared, rồi cộng bias từ constant.
- Key snippet:
```cuda
// cooperative loading tile+halo
int tid = ty * blockDim.x + tx;
for (int idx = tid; idx < BLOCK_H * BLOCK_W; idx += num_threads) {
    int i = idx / BLOCK_W, j = idx % BLOCK_W;
    int h_in = row_start + i, w_in = col_start + j;
    smem[i][j] = (in-bound) ? input[idx4(...)] : 0.0f;
}
__syncthreads();

// unroll K=3
#pragma unroll
for (int i = 0; i < 3; ++i)
#pragma unroll
for (int j = 0; j < 3; ++j)
    value += smem[...+i][...+j] * weight[idx4(c_out,c_in,i,j,...)];
```
2. Conv2D Backward Input (dX): Shared halo
- Kỹ thuật áp dụng: Load `dY` tile kèm halo vào shared memory `s_dY[TILE_H+2][TILE_W+2]`.
- Vì sao giúp nhanh hơn: Mỗi phần tử dY trong tile được tái sử dụng nhiều lần, giúp giảm global loads.
- Cách triển khai:
  - Mỗi block xử lý 1 tile của dX (theo n, c_in).
  - Với mỗi c_out: load shared dY, sync, compute accumulate.
- Key snippet:
```cuda
__shared__ float s_dY[TILE_H + 2][TILE_W + 2];

// load dY with halo
int ho = base_h + sy - 1;
int wo = base_w + sx - 1;
s_dY[sy][sx] = (in-bound) ? dY[idx4(n,c_out,ho,wo,...)] : 0.0f;
__syncthreads();
```
3. Conv2D Backward Weight (dW): Block-level reduction trước `atomicAdd` (giảm atomic contention)
- Kỹ thuật áp dụng:
   - Mỗi thread tính `dw[kh][kw`] cục bộ (register).
   - Ghi vào buffer shared `s_dw[k_idx][tid]` rồi tree-reduction trong block.
   - Chỉ thread 0 `atomicAdd` cho mỗi kernel element `(kh,kw)`.
- Vì sao giúp nhanh hơn:
  - Nếu mỗi thread atomic, số atomic cực lớn dẫn đến contention nặng.
  - Reduction trong shared rẻ hơn nhiều so với atomic trên global.
  - Giảm số atomic từ #threads xuống 1 atomic / (kh,kw) / block.
- Cách triển khai:
  - Cooperative loading cả input (tile+halo) và dY (tile) vào shared.
  - Tính local dw theo tile, sau đó reduce.
- Key snippet:
```cuda
// store local dw to shared then reduce
int k_idx = kh * 3 + kw;
s_dw[k_idx][tid] = dw[kh][kw];
__syncthreads();

for (int s = num_threads / 2; s > 0; s >>= 1) {
    if (tid < s) s_dw[k_idx][tid] += s_dw[k_idx][tid + s];
    __syncthreads();
}

// only 1 atomic per (kh,kw) per block
if (tid == 0) atomicAdd(&dW[idx4(c_out,c_in,kh,kw,...)], s_dw[k_idx][0]);

```
4. Conv2D Backward Bias (dB): Warp-level reduction bằng shuffle
- Kỹ thuật áp dụng:
  - Grid-stride loop cộng dY theo channel c.
  - Reduce trong warp bằng `__shfl_down_sync`.
  - Mỗi warp ghi 1 giá trị vào `warp_sums[]`, rồi warp 0 reduce lần cuối.
- Vì sao giúp nhanh hơn:
  - Shuffle giảm phụ thuộc shared memory và giảm số lần `__syncthreads()`.
  - Phù hợp với phép cộng reduction dạng tuyến tính.
- Key snippet:
```cuda
#pragma unroll
for (int offset = 16; offset > 0; offset >>= 1)
    sum += __shfl_down_sync(0xffffffff, sum, offset);
```
#### Results:
- Tổng thời gian huấn luyện: 2946.121s (khoảng 49 phút 6 giây)
- Thời gian huấn luyện trung bình mỗi epoch: khoảng 147.3s (2 phút 27 giây)
- Tăng tốc so với version 1: nhanh hơn khoảng 1.19 lần so với version 1
- Sample reconstructed images
![Ảnh mẫu reconstructed bởi GPU version 2](https://drive.google.com/uc?export=view&id=1CgsqohZwarREM4Fmc1W5ynOcQRsZEYL1)
- So sánh profiling version 1 và 2:  
Version 1: 
![Ảnh profiling của GPU version 1](https://drive.google.com/uc?export=view&id=14WZFvBhJjrJnsWoqetQK1qLVNGBz-oJw)  
Version 2:
![Ảnh profiling của GPU version 2](https://drive.google.com/uc?export=view&id=1r-vtGWFcIMvUWr5YUHMtirbtFIf4pIxS)  
V2 giảm thời gian trung bình mỗi lần gọi của cả 3 kernel Conv2D, chứng tỏ tối ưu (cooperative loading/unroll/reduction-before-atomic, v.v.) có tác dụng trực tiếp lên compute chính. 
#### Phân tích:
- Tối ưu này hiệu quả vì: Phần tối ưu trong version này tập trung trực tiếp vào 3 kernel chiếm nhiều thời gian chạy nhất. Các kỹ thuật như cooperative loading, unroll K=3 và đặc biệt block-level reduction trước atomicAdd giúp:
  - Giảm số lần truy cập global memory và tăng tái sử dụng dữ liệu trong block.
  - Giảm overhead vòng lặp nhờ unroll.
  - Giảm mạnh atomic contention ở backward_weight (ít atomic hơn nên ít tranh chấp hơn).  
Tuy nhiên, vì Conv2D vẫn là workload chính nên mức tăng tổng thể quanh 15-20%.
- Bottleneck lớn nhất vẫn là conv2d_backward_weight (khoảng 38%), vì đây là kernel có chi phí cao và dễ bị giới hạn bởi atomic/giảm tốc do truy cập bộ nhớ và tích lũy gradient. Sau đó là conv2d_forward và conv2d_backward_input (khoảng 30% mỗi kernel).
#### Key Takeaways:
- Bài học rút ra được từ lần tối ưu này:
  - Profiling rất quan trọng trong việc xác định yếu tố để tối ưu. Trước khi tối ưu, cần xác định đúng kernel chiếm thời gian lớn. Kết quả cho thấy 3 kernel Conv2D chiếm ~97% GPU time, nên tối ưu vào đây mới tạo ra cải thiện tổng thể rõ rệt.
  - Giảm “memory traffic” + tăng reuse là chìa khóa: Cooperative loading và tái sử dụng tile trong shared memory giúp giảm đọc global lặp lại, cải thiện throughput đáng kể ở forward/backward_input.
  - Atomic contention là “kẻ giết hiệu năng”: Backward weight thường tốn nhất vì tích lũy gradient dễ gây tranh chấp. Việc block-level reduction trước atomicAdd giúp giảm contention → giảm thời gian kernel rõ rệt.
- Khả năng áp dụng cho các bài toán khác:
  - Các kỹ thuật này áp dụng tốt cho các phép toán dạng convolution và các bài toán có tái sử dụng dữ liệu theo vùng (image filtering, conv3d, correlation, sliding window).
  - Block-level reduction trước atomic áp dụng cho mọi bài toán có kiểu “nhiều thread cộng dồn vào một biến” (histogram, gradient accumulation, sum pooling, scatter-add), giúp giảm contention rõ rệt.
  - Warp shuffle reduction áp dụng cho các phép reduce/sum theo chiều kênh hoặc theo batch, phù hợp cho bias gradient, layer norm stats, tổng loss, metrics…

### 2.5. Tích hợp SVM:
#### Chi tiết triển khai:
- Trích xuất đặc trưng từ Encoder:
    - Quá trình trích xuất đặc trưng gồm 2 bước:
        - Bước 1: Chúng em tải lại trọng số của mô hình đã huấn luyện. Thực hiện lại phần kiến trúc Encoder (từ lớp Input đến lớp Latent Space) và loại bỏ hoàn toàn phần Decoder. Điều này đảm bảo mạng nơ-ron hoạt động như một bộ trích xuất đặc trưng thuần túy.
        - Bước 2: Mỗi ảnh đầu vào x (kích thước 32×32×3) được đưa qua các lớp tích chập (Conv2D) và giảm mẫu (MaxPooling). Tại lớp cuối cùng của Encoder, dữ liệu được cô đọng lại thành một tensor 3 chiều có kích thước 8×8×128.
- Tích hợp LIBSVM: 
    - Thay vì nhúng trực tiếp thư viện LIBSVM vào mã nguồn C có thể gây phức tạp trong việc biên dịch và quản lý bộ nhớ, nhóm thực hiện chiến lược tương tác gián tiếp thông qua tệp tin. Quy trình được chia thành hai bước:
        - Bước 1: Sử dụng Encoder đã huấn luyện để trích xuất đặc trưng và ghi ra tệp văn bản theo định dạng chuẩn của LIBSVM.
        - Bước 2: Sử dụng thư viện LIBSVM (thông qua CLI) để đọc tệp dữ liệu này và thực hiện huấn luyện.
- Lựa chọn siêu tham số: 
    - Loại SVM: C-SVC (C-Support Vector Classification).
    - Loại kernel: Linear kernel (Nhân tuyến tính).
    - Tham số phạt: C = 1.0.
- Các đoạn mã chính:
    1) Trích xuất đặc trưng dùng để huấn luyện:
    ```c
    // Lặp qua từng batch
    for (int b = 0; b < num_batches_train; ++b) {
        // Copy ảnh vào bộ đệm
        // Thực hiện encoder
        cpu_extract_features(&autoencoder, h_batch, cur_bs, h_latent);
        // Ghi file theo format LIBSVM
        for (int i = 0; i < cur_bs; ++i) {
            int idx = start + i;
            int label = data.train_labels[idx];
            const float* feat = h_latent + i * AE_LATENT_DIM;
            write_svm_line(f_train, label, feat, AE_LATENT_DIM);
        }
    }
    ```
    2) Mã huấn luyện SVM: Phiên bản GPU optimization 2.
    ```python
    !./svm-train -s 0 -t 0 -c 1.0 \
    "../opt2/train_svm.txt" \
    ../opt2/model_ae_svm
    ```
#### Kết quả:
- Thời gian trích xuất đặc trưng (50K train + 10K test):
    - Phiên bản CPU: Khoảng 3 tiếng (cho 10K ảnh train và 2K ảnh test).
    - Phiên bản GPU naive: Khoảng 4 phút 30 giây.
    - Phiên bản tối ưu hoá GPU (Phiên bản 1): Khoảng 4 phút.
    - Phiên bản tối ưu hoá GPU (Phiên bản 2): Khoảng 3 phút 40 giây.
- Thời gian huấn luyện SVM:
    - Phiên bản CPU huấn luyện khoảng 1 tiếng.
    - Mỗi phiên bản GPU huấn luyện khoảng 7 tiếng.
- Độ chính xác phân loại trên tập dữ liệu thử nghiệm: 
    - Phiên bản CPU: 56.65%, do nhóm em chỉ huấn luyện SVM với 10 ngàn ảnh nên sẽ không đạt được chính xác mong đợi là 60-65%.
    - Phiên bản GPU naive: 63.63%.
    - Phiên bản tối ưu hoá GPU (Phiên bản 1): 63.96%.
    - Phiên bản tối ưu hoá GPU (Phiên bản 2): 64.25%.
- Phân tích độ chính xác theo từng lớp:

    1) Phiên bản CPU:

    |ID lớp|Độ chính xác (%)|
    |------|----------------|
    |0|11.8|
    |1|13.4|
    |2|7.9|
    |3|8.7|
    |4|8.6|
    |5|6.8|
    |6|15.6|
    |7|13|
    |8|15.6|
    |9|11.9|

    2) Phiên bản GPU naive:

    |ID lớp|Độ chính xác (%)|
    |------|----------------|
    |0|69.4|
    |1|71.2|
    |2|48.5|
    |3|45.3|
    |4|60.2|
    |5|54.3|
    |6|75.8|
    |7|66.8|
    |8|75|
    |9|69.8|

    3) Phiên bản tối ưu hoá GPU (Phiên bản 1):

    |ID lớp|Độ chính xác (%)|
    |------|----------------|
    |0|67.3|
    |1|71.7|
    |2|51|
    |3|45.2|
    |4|60.2|
    |5|55|
    |6|74.1|
    |7|69|
    |8|76|
    |9|70.1|

    4) Phiên bản tối ưu hoá GPU (Phiên bản 2):

    |ID lớp|Độ chính xác (%)|
    |------|----------------|
    |0|68.2|
    |1|72.5|
    |2|49.7|
    |3|47|
    |4|60.8|
    |5|54.3|
    |6|76|
    |7|68.3|
    |8|74.6|
    |9|71.1|

- Confusion matrix:
1) Phiên bản CPU:

![Confusion matrix cho CPU](https://drive.google.com/uc?export=view&id=1JXa1PKPK55oXImVMVrgwEAgf2hObp0Gb)

2) Phiên bản GPU naive:

![Confusion matrix cho GPU Naive](https://drive.google.com/uc?export=view&id=12FQrKBFRRpnVDVCJ71m5XZtWDsiOYmRm)

3) Phiên bản tối ưu hoá GPU (Phiên bản 1):

![Confusion matrix cho GPU V1](https://drive.google.com/uc?export=view&id=1YgcJr-rr7cHVnLTmdx84bUOEsOpu13de)

4) Phiên bản tối ưu hoá GPU (Phiên bản 2):

![Confusion matrix cho GPU V2](https://drive.google.com/uc?export=view&id=1EhF_rA_7LW-WbjapH_84-jbJ9u3mRrPK)

#### Phân tích:
- Những lớp dễ phân loại và khó phân loại nhất:
    - Kết quả trên tập kiểm tra cho thấy mô hình hoạt động rất hiệu quả trên nhóm đối tượng nhân tạo (máy bay, ô tô) và các lớp có đặc trưng thị giác nổi bật như lớp 6 (ếch) và lớp 8 (tàu thủy). Sự vượt trội này xuất phát từ tính đặc thù của dữ liệu. Ví dụ, ảnh tàu thủy thường gắn liền với nền xanh đồng nhất (nước hoặc bầu trời), ít nhiễu nền, giúp Autoencoder dễ dàng cô lập đối tượng. Tương tự, ếch có hình thái cơ thể tròn trịa và tư thế ngồi đặc trưng, khác biệt hoàn toàn so với cấu trúc bốn chân của các loài thú khác, giúp mô hình dễ dàng phân tách . 
    - Tuy nhiên, độ chính xác bị giảm sút đáng kể ở các lớp động vật, đặc biệt là lớp 2 (chim) và lớp 3 (mèo) với độ chính xác chỉ đạt ngưỡng 50%. Cho thấy hai lớp này sẽ khó phân loại nhất do cấu trúc sinh học phức tạp của chúng có thể gây nhầm lẫn sang thực thể khác.
    
    **Kết luận:** Điều đó cho thấy, mặc dù SVM phân loại tốt các đối tượng có hình học cố định, nhưng việc trích xuất đặc trưng cho các đối tượng sinh học phức tạp vẫn là một thách thức, gợi ý rằng mô hình cần huấn luyện lâu hơn hoặc cần kiến trúc Autoencoder sâu hơn để bắt được các chi tiết tinh tế này.
- Ma trận nhầm lẫn tiết lộ những sai lầm của mô hình SVM trong quá trình phân loại nhãn, cụ thể như sau:
    - Phân tách rõ ràng giữa Động vật và Phương tiện: Ma trận cho thấy các lỗi phân loại sai không phân bố ngẫu nhiên mà có xu hướng co cụm. Mô hình hiếm khi nhầm lẫn giữa nhóm phương tiện (Máy bay, ô tô, tàu, xe tải) và nhóm động vật. Cho thấy Autoencoder đã học thành công các đặc trưng toàn cục, đủ để SVM có thể phân tách hai siêu lớp này.
    - Sự nhập nhằng do tương đồng thị giác: Đa số các trường hợp dự đoán sai xảy ra giữa các cặp lớp có cấu trúc hình học tương tự nhau. Cụ thể:
        - Ở các phiên bản, cho thấy rằng mô hình phần lớn nhầm lẫn giữa lớp chó và mèo. Vì chúng có các đặc điểm giống nhau, ví dụ như sự tương đồng về kích thước, cấu trúc 4 chân và kết cấu lông.
        - Ngoài ra, mô hình cũng thường xuyên nhầm lẫn giữa lớp ô tô và lớp xe tải do chúng đều cùng có bánh xe và hình dáng hộp.
        - Điều này chỉ ra giới hạn của mô hình là các đặc trưng trong không gian tiềm ẩn (Latent Space) chưa đủ độ chi tiết để phân biệt các khác biệt nhỏ về mặt cấu trúc sinh học.
    **Kết luận:** Phân tích ma trận nhầm lẫn cho thấy mô hình đã đạt được sự trừu tượng hóa cấp cao, thể hiện qua khả năng phân tách rạch ròi giữa hai nhóm đối tượng lớn là 'Phương tiện' và 'Động vật' dựa trên các cấu trúc hình học tổng quát. Tuy nhiên, các biểu diễn trong không gian tiềm ẩn chưa đủ chi tiết để giải quyết bài toán phân biệt các đối tượng có độ tương đồng thị giác cao (như kết cấu lông hay hình dáng cơ thể giữa các loài thú), dẫn đến sự nhập nhằng trong các dự đoán nội bộ nhóm.
- Độ chính xác so với kỳ vọng: Trong phần mô tả, nêu rõ là kỳ vọng độ chính xác 60-65%. Ở đây tất cả các phiên bản đều đạt được độ chính xác kỳ vọng là 63-64% (ngoại trừ CPU không đạt do huấn luyện dữ liệu ít hơn).
#### Những điểm chính:
- Chất lượng của các đặc trưng mô hình đã học: Kết quả đạt được từ việc huấn luyện mô hình cho thấy rằng những đặc trưng mà Autoencoder đã học được rất tốt. Từ đó giúp mô hình có khả năng phân loại tốt, và đạt được hiệu quả mong đợi.
- Effectiveness of two-stage approach

## 3. Comprehensive Performance Analysis
### 3.1. Performance Comparison Across All Phases:
#### Table Format:
| Phase        | Training Time | Speedup (vs CPU) | Incremental Speedup | Memory Usage | Key Optimization        |
|-------------|---------------|------------------|----------------------|-------------|-------------------------|
| CPU Baseline| 9750s (1000 ảnh)        | 1.0×             | -                    | -           | -                       |
| GPU Basic   | 6091s          | 80.0×            | 80.0×                | -      | Parallelization         |
| GPU Opt v1  | 3507s           | 139.0×            | 1.73×                 | -      | Shared memory tiling + Constant-memory bias           |
| GPU Opt v2  | 2946s           | 165.0×            | 1.19×                | -      | Cooperative loading + Loop unrolling + Block reduction trước atomic + Warp shuffle reduction |

#### Visualizations:
- Bar chart showing training time across phases
- Line graph showing cumulative speedup

## 4. Bài học và khó khăn trong quá trình thực hiện:
### 4.1. Những kỹ thuật quan trọng:
- Lập trình CUDA:
  - Hiểu rõ cách tổ chức kernel theo mô hình grid/block/thread, đặc biệt là chiến lược ánh xạ (x,y) cho không gian (h,w) và z cho (batch, channel) để xử lý tensor NCHW hiệu quả.
  - Nắm vững các vấn đề khi làm việc với CUDA như: quản lý truy cập bộ nhớ, tránh ghi đè (race condition) và quản lý lỗi bằng CHECK_CUDA.
  - Hiểu được tầm quan trọng của việc đồng bộ các hoạt động được thực thi trên GPU để đảm bảo thu được kết quả đúng và hiệu quả.
- Deep Learning: 
  - Hiểu được kiến trúc của mô hình Autoencoder, cách mà mô hình hoạt động để học và trích xuất được đặc trưng không cần nhãn.
  - Biết được các hoạt động của các lớp trong mô hình (Conv, Pool, Upsample), cách hoạt động của encoder-decoder.
- Tối ưu hiệu suất:
  - Thiết lập được baseline để tối ưu: Trước khi thực hiện tối ưu cần phải chính xác và có mốc đo để so sánh.
  - Thấy rõ được bottleneck của baseline, từ đó đưa ra các chiến lược tối ưu thích hợp để đạt được hiệu suất tốt nhất.  
### 4.2. Khó khăn chính và giải pháp:
#### - Khó khăn 1: Chuyển code từ CPU sang GPU vẫn giữ đúng logic và độ chính xác
- Vấn đề: Khi chuyển đổi từng layer sang GPU, rất dễ sai ở phần chỉ số (index), mapping thread với output, chỉ cần lệch 1 chỉ số là output sai toàn bộ.
- Giải pháp: 
  - Có test để verify kết quả so với CPU theo từng kernel.
  - Dùng một layout index thống nhất là idx4().
- Bài học: Cần phải đảm bảo chính xác từng baseline viết bằng CPU, có test để verify kết quả của GPU và CPU, sau đó mới tập trung vào tối ưu.
#### - Khó khăn 2: Train trên số ít epoch khiến mô hình hội tụ không tốt.
- Vấn đề: Do hạn chế về mặt tài nguyên, nhóm chỉ có thể train trên số lượng ít epoch. Có trường hợp đạt loss thấp nhưng khi extract_features và đưa sang bộ phân loại SVM thì accuracy chỉ đạt quanh 10%.
- Giải pháp:
  - Xem xét lại phần khởi tạo trọng số, điều chỉnh khoảng khởi tạo tốt hơn.
  - Tăng learning rate
- Bài học:
  - Reconstruction loss thấp không đảm bảo latent representation sẽ tốt, vẫn có trường hợp tệ là đặc trưng được trích xuất không mang nhiều thông tin hỗ trợ phân loại lớp.
  - Khởi tạo trọng số và learning rate trong vài trường hợp có ảnh hưởng mạnh đến hiệu suất mô hình. Khởi tạo không hù hợp có thể khiến mô hình hội tụ kém, tạo ra latent không có ý nghĩa phân loại. Tương tự, learning rate quá nhỏ khiến mô hình học chậm, quá lớn lại đi lệch đi điểm tối ưu.

## 5. Kết luận và hướng nghiên cứu trong tương lai:
### 5.1. Tóm tắt đồ án:
- Trong đồ án này, nhóm đã:
  - Xây dựng pipeline huấn luyện Autoencoder trên CPU và GPU với các lớp chính gồm: Conv2D, ReLU, MaxPool 2x2, UpSample 2x2, MSE loss, SGD update.
  - Cài đặt được hai phiên bản tối ưu cho GPU gồm phiên bản 1 tập trung vào chiến lược Shared Memory Tiling, còn phiên bản 2 áp dụng nhiều chiến lược cho các hàm Conv2D chiếm dụng nhiều thời gian.
  - Đưa các kết quả trích xuất đặc trưng thu được thông qua các mô hình huấn luyện cho bộ phân loại SVM và đạt được độ chính xác như mong muốn.
- Bảng tóm tắt các chỉ số hiệu suất cuối cùng:
| Phase        | Training Time | Speedup (vs CPU) | Accuracy |
|-------------|---------------|------------------|----------------------|
| CPU Baseline| 9750s (1000 ảnh)        | 1.0×     |                 | 
| GPU Basic   | 6091s          | 80.0×           |                 | 
| GPU Opt v1  | 3507s           | 139.0×         |                 |  
| GPU Opt v2  | 2946s           | 165.0×        |                  |
- Đã đạt được các mục tiêu liên quan đến độ chính xác của việc phân loại, hiệu suất của các phiên bản GPU giảm dần thông qua các chiến lược tối ưu.
### 5.2. Thành tựu chính:
Nêu bật những kết quả tốt nhất:
- Tốc độ tăng tốc tối đa đạt được: 165.0x so với GPU
- Độ chính xác khi phân loại: 
- Kỹ thuật tối ưu hóa thành công nhất: 
  - Block-level reduction trước atomicAdd cho backward_weight (giảm atomic contention).
  - Cooperative loading + shared memory reuse + unroll K=3 cho forward/backward_input.
  - Warp shuffle reduction cho gradient bias.
- Thành thạo các kỹ thuật:
  - Thiết kế CUDA kernels (forward/backward), quản lý bộ nhớ, tối ưu truy cập bộ nhớ.
  - Profiling & phân tích bottleneck (nvprof), đánh giá cải tiến bằng số liệu.
  - Phân tích kết quả phân loại thu được.
### 5.3. Hạn chế:
- Các điểm nghẽn hiệu suất hiện tại: conv2d_backward_weight vẫn là kernel nặng nhất (~38%), sau đó là forward/backward_input (~30% mỗi cái).
- Hạn chế về độ chính xác: Do hạn chế về số lớp của Autoencoder cũng như bộ phân loại SVM, kết quả độ chính xác phân loại thu được chưa hơn 65%
- Các ràng buộc khi triển khai: Hạn chế về số lượng epoch, lượng dữ liệu và đặc biệt là tài nguyên GPU thực thi.
### 5.4. Cải tiến trong tương lai:
- Có thể tìm cách cải tiến thêm cho các hàm Conv2D backward.
- Có thể áp dụng thư viện hỗ trợ cho CUDA để tối ưu thêm.
